## Kütüphanelerin İmport Edilmesi

Bu bölümde, modelin eğitimi ve değerlendirilmesi için gerekli olan Python kütüphaneleri import edilir.


In [1]:
!pip install transformers


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install openpyxl --upgrade


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
import tensorflow as tf
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
#config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

D0311 13:18:25.860886829    3268 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0311 13:18:25.860913746    3268 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0311 13:18:25.860917581    3268 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0311 13:18:25.860920781    3268 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0311 13:18:25.860923536    3268 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0311 13:18:25.860926235    3268 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0311 13:18:25.860929050    3268 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0311 13:18:25.

## Veri Setinin Yüklenmesi ve Ön İşleme

Bu bölümde, veri seti yüklenir ve modelin eğitimi için uygun hale getirilir. Ön işleme adımları arasında veri temizleme, tokenizasyon ve vektörleştirme bulunabilir.


In [4]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 200
steps_per_epoch = 60000 // batch_size
epochs = 2

In [5]:
import pandas as pd
data1 = pd.read_excel('/kaggle/input/snli-turkish/snli_turkish.xlsx')  
data2 = pd.read_excel('/kaggle/input/snli-turkish/snli_turkish1.xlsx')  
data3 = pd.read_excel('/kaggle/input/snli-turkish/snli_turkish2.xlsx')  
data4 = pd.read_excel('/kaggle/input/snli-turkish/snli_turkish3.xlsx')  
data5 = pd.read_excel('/kaggle/input/snli-turkish/snli_turkish4.xlsx')  
real_data  = pd.read_excel('/kaggle/input/snli-turkish/log_df.xlsx')

labels = ["contradiction", "entailment", "neutral"]

In [6]:
real_data.columns = ["Öncül"," hipotez"," etiket"]

for col in data5.columns:
    print(col)

Öncül
 hipotez
 etiket


In [7]:
data = pd.concat([data1, data2, data3, data4, data5, real_data],ignore_index=True)

In [8]:
display(data[500000:])

,Öncül,hipotez,etiket
500000,"Konya'da, kız kardeşi E.B.'ye (16) cinsel isti...","Konya'da, kız kardeşi E.B.'ye (16) cinsel isti...",0
500001,MHP Eyüp eski İlçe Başkanı Köksal Kaçmaz'ın vu...,"Konya'da, kız kardeşi E.B.'ye (16) cinsel isti...",2
500002,MHP Eyüp eski İlçe Başkanı Köksal Kaçmaz'ın vu...,MHP Eyüp eski İlçe Başkanı Köksal Kaçmaz'ın vu...,0
500003,Merkez Bankası’nın faizi yüzde 20’ye çıkarması...,"Konya'da, kız kardeşi E.B.'ye (16) cinsel isti...",2
500004,Merkez Bankası’nın faizi yüzde 20’ye çıkarması...,MHP Eyüp eski İlçe Başkanı Köksal Kaçmaz'ın vu...,2
...,...,...,...
500413,"Cumhurbaşkanı Erdoğan: ""Gözümüzü 2053'e diktik.""","Yozgat'ta 12 kişinin yaşamını yitirdiği, 18 ki...",2
500414,"Cumhurbaşkanı Erdoğan: ""Gözümüzü 2053'e diktik.""","Cumhurbaşkanı Erdoğan: ""Hayat pahalılığı sorun...",2
500415,"Cumhurbaşkanı Erdoğan: ""Gözümüzü 2053'e diktik.""","Cumhurbaşkanı Erdoğan: ""Hayat pahalılığı sorun...",2
500416,"Cumhurbaşkanı Erdoğan: ""Gözümüzü 2053'e diktik.""","#SONDAKİKA | Türkiye, Yunanistan'a yangınlarla...",2


In [9]:
import re

def process_text(text):
    # Noktalama işaretlerini kaldırmak için regex kullanıyoruz

    no_punct_spaces_text = re.sub(r'[^\w\s]', ' ', str(text))
    
    # Ardışık boşlukları tek bir boşlukla değiştiriyoruz
    final_text = re.sub(r'\s+', ' ', no_punct_spaces_text).strip()
    
    return final_text

# DataFrame'deki her metin için işlemleri uyguluyoruz
data['Öncül'] = data['Öncül'].apply(process_text)
data[' hipotez'] = data[' hipotez'].apply(process_text)

In [10]:
display(data[500000:])

,Öncül,hipotez,etiket
500000,Konya da kız kardeşi E B ye 16 cinsel istismar...,Konya da kız kardeşi E B ye 16 cinsel istismar...,0
500001,MHP Eyüp eski İlçe Başkanı Köksal Kaçmaz ın vu...,Konya da kız kardeşi E B ye 16 cinsel istismar...,2
500002,MHP Eyüp eski İlçe Başkanı Köksal Kaçmaz ın vu...,MHP Eyüp eski İlçe Başkanı Köksal Kaçmaz ın vu...,0
500003,Merkez Bankası nın faizi yüzde 20 ye çıkarması...,Konya da kız kardeşi E B ye 16 cinsel istismar...,2
500004,Merkez Bankası nın faizi yüzde 20 ye çıkarması...,MHP Eyüp eski İlçe Başkanı Köksal Kaçmaz ın vu...,2
...,...,...,...
500413,Cumhurbaşkanı Erdoğan Gözümüzü 2053 e diktik,Yozgat ta 12 kişinin yaşamını yitirdiği 18 kiş...,2
500414,Cumhurbaşkanı Erdoğan Gözümüzü 2053 e diktik,Cumhurbaşkanı Erdoğan Hayat pahalılığı sorunun...,2
500415,Cumhurbaşkanı Erdoğan Gözümüzü 2053 e diktik,Cumhurbaşkanı Erdoğan Hayat pahalılığı sorunun...,2
500416,Cumhurbaşkanı Erdoğan Gözümüzü 2053 e diktik,SONDAKİKA Türkiye Yunanistan a yangınlarla müc...,2


In [11]:
train_df = data

In [12]:
print(train_df[" etiket"].value_counts())

 etiket
 2    166905
 0    166739
 1    166093
-1       681
Name: count, dtype: int64


In [13]:
train_df = (
    train_df[train_df[" etiket"] != -1]
    .sample(frac=1.0, random_state=42)
    .reset_index(drop=True)
)

In [14]:
print(train_df[" etiket"].value_counts())

 etiket
2    166905
0    166739
1    166093
Name: count, dtype: int64


In [15]:
y_train = tf.keras.utils.to_categorical(train_df[" etiket"], num_classes=3)

In [16]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(
            "dbmdz/bert-base-turkish-128k-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

In [17]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

strategy = tf.distribute.TPUStrategy(tpu)

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFAutoModel.from_pretrained("dbmdz/bert-base-turkish-128k-uncased")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    bert_output = bert_model(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    sequence_output = bert_output.last_hidden_state
    pooled_output = bert_output.pooler_output
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(256, return_sequences=True)
    )(sequence_output)
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    x = tf.keras.layers.Dense(128, activation="relu")(dropout)
    dropout = tf.keras.layers.Dropout(0.3)(x)
    output = tf.keras.layers.Dense(3, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        steps_per_execution = 50,
        loss="categorical_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU

Some layers from the model checkpoint at dbmdz/bert-base-turkish-128k-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Strategy: <tensorflow.python.distribute.tpu_strategy.TPUStrategyV2 object at 0x79eebf80ce80>
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  184345344   ['input_ids[0][0]',              
 

In [18]:


train_data = BertSemanticDataGenerator(
    train_df[["Öncül", " hipotez"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)

In [19]:
history = model.fit(
    train_data,
    epochs=0,
    steps_per_epoch=steps_per_epoch,
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [20]:
model.fit(
    train_data,
    epochs=0,
    steps_per_epoch=steps_per_epoch,
)

In [22]:
import keras

with strategy.scope():
    model = tf.keras.models.load_model('/kaggle/input/keras-80acc/keras.model', custom_objects={"TFBertModel": transformers.TFAutoModel.from_pretrained("dbmdz/bert-base-turkish-128k-uncased")})
    
    for l in model.layers:
        if l.name == "tf_bert_model":
            l.trainable = True
        print(l.name, l.trainable)    
        
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-5),
        steps_per_execution = 50,
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    model.summary()
    
model.fit(
    train_data,
    epochs=0,

)

Some layers from the model checkpoint at dbmdz/bert-base-turkish-128k-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


input_ids True
attention_masks True
token_type_ids True
tf_bert_model True
bidirectional True
global_average_pooling1d True
global_max_pooling1d True
concatenate True
dropout_37 True
dense True
dropout_38 True
dense_1 True
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                     

/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


<a href="/kaggle/working/keras.model"> Download File </a>

In [23]:
def check_similarity(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )
    proba = model.predict(test_data[0])[0]
    idx = np.argmax(proba)
    proba = f"{proba[idx]: .2f}%"
    pred = idx
    return pred, proba

In [ ]:
model.save("keras.model")

In [24]:
sentence1 = "İstanbul’da trafik kurallarına uymayan ve agresif tavırlar sergileyen taksiciye arka arkaya 2 ceza kesildi.."
sentence2 = "Şehit Eren Bülbül'ün annesi sezonun ilk maçında Trabzonspor stadyumuna geldi"
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
2024-03-11 13:26:30.338949: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2024-03-11 13:26:31.140156: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


1/1 [==============================] - 17s 17s/step


(2, ' 0.98%')